#### Anomaly Detection - Imports and Setup

In [ ]:
%reload_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Lambda, Dropout, SimpleRNN, Dense, LSTM, RepeatVector, Input, TimeDistributed, concatenate
from keras import regularizers
from keras.utils import plot_model

import IPython, IPython.display, os, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

print(f"Tensorflow Version {tf.__version__}, Keras Vesion: {keras.__version__}")

In [ ]:
import ts_utils
import ts_plot_utils
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

# STEP 1: = >Lets just read the first few columns for testing
df = pd.read_csv("../data/processminer-rare-event-mts.csv.zip", sep=';', usecols=range(59))
split = int (.8 * len(df) )
df_scaled_trn = df [df.columns[2:] ][0:split ]
df_scaled_tst = df [df.columns[2:] ][split: ]

scaler = MinMaxScaler()
df_scaled_trn = pd.DataFrame(scaler.fit_transform(df_scaled_trn), columns=df_scaled_trn.columns)
df_scaled_tst = pd.DataFrame(scaler.transform(df_scaled_tst), columns=df_scaled_tst.columns)


#  STEP 2: => Create window

input_slice  = slice(0, len(df_scaled_trn.columns) )
label_slice  = input_slice
window_len   = 5
ouput_len    = 1
batch_size   = 64

inp_feat_len    = input_slice.stop - (input_slice.start or 0)
ouput_feat_len  = label_slice.stop - (label_slice.start or 0)

ds_trn        = tf.data.Dataset.from_tensor_slices(df_scaled_trn[df_scaled_trn.columns[input_slice]])
ds_tst        = tf.data.Dataset.from_tensor_slices(df_scaled_tst[df_scaled_trn.columns[input_slice]])
window_trn    = ts_utils.window(ds_trn, window_len, ouput_len, label_slice, batch_size=batch_size,)
window_tst    = ts_utils.window(ds_tst, window_len, ouput_len, label_slice, batch_size=batch_size )
window_trn100 = ts_utils.window(ds_trn, window_len, ouput_len, label_slice, batch_size=100000  )
window_tst100 = ts_utils.window(ds_tst, window_len, ouput_len, label_slice, batch_size=100000  )

df_scaled_trn
#for w in window_trn.take(1): print(f'{w[0]} \n\n {w[1]}' )

### Create model forecasting

In [ ]:
from keras.utils import plot_model
performance = {}
models = []

# Linear Model
linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :])
    ] + ts_utils.getCommonLayer(ouput_len, ouput_feat_len),
    name = "Linear"
)
models.append(linear_model)
ts_utils.compile_fit(models[-1])
models[-1].fit(window_trn.take(1), epochs=1)


In [ ]:
dim = 256

# Create Autoencoder Layer
input_layer = Input(shape=(window_len, inp_feat_len), dtype='float32', name='input')
memory_layer = LSTM(dim, return_sequences=True)(input_layer)
memory_layer = LSTM (dim//2, return_sequences=False)(memory_layer)
repeated_lyr = RepeatVector(window_len)(memory_layer)
memory_layer = LSTM (dim//2, return_sequences=True)(repeated_lyr)
memory_layer = LSTM (dim,  return_sequences=True)(memory_layer)
decoded_inputs = TimeDistributed(Dense(units=inp_feat_len, activation='linear'))( memory_layer)

dropout_input = Dropout(0.2)(input_layer)
concat_layer = concatenate([dropout_input, decoded_inputs])
memory_layer = LSTM(units=dim, 
                    kernel_regularizer = regularizers.l1_l2(l1= 0, l2= 0), 
                    recurrent_regularizer = regularizers.l1_l2(l1= 0, l2= 0), 
                    return_sequences=False)(concat_layer)

# => Note this is same as getCommonLayer(ouput_len, ouput_feat_len, memory_layer)
#
preds = Dense(units=ouput_feat_len*ouput_len)(memory_layer)
preds = Dense(units=ouput_feat_len*ouput_len, activation='linear')(preds)
preds = tf.keras.layers.Reshape([ouput_len, ouput_feat_len])(preds)
umodel = Model(input_layer, preds, name="Uber")

models.append(umodel)
ts_utils.compile_fit(models[-1])
models[-1].fit(window_trn.take(1), epochs=1)


In [ ]:
umodel.layers[-2].set_weights(linear_model.layers[-2].weights)
ts_utils.compile_fit(models[-1])
models[-1].fit(window_trn.take(1), epochs=1)

In [ ]:
# LSTM Model
lstm_model = tf.keras.Sequential([
        # Shape [batch, time, features] => [batch, lstm_units].
        # Adding more `lstm_units` just overfits more quickly.

        tf.keras.layers.LSTM(32, return_sequences=False)
    ]+ ts_utils.getCommonLayer(ouput_len, ouput_feat_len),
    name = "LSTM"
)
models.append(lstm_model)

In [ ]:
#%%script echo
for i, model in enumerate(models):
    print(f"Now Compiling model: {model.name} => {i+1}/{len(models)} models")
    history = ts_utils.compile_fit(model, window_trn, window_tst, epochs=50, patience=3, verbose=1)
    IPython.display.clear_output()

IPython.display.clear_output()
# Plot graphs
#performance={}
performance = ts_plot_utils.plot_performance(models, window_trn100, window_tst100, performance=performance, reeval=1)

In [ ]:
IPython.display.clear_output()

for l in history.history:
    plt.plot(history.history[l], label=f"{l}")
plt.title("History of Losses")
plt.legend()

### Anomaly Score - precision/Recall etc.

In [ ]:
model     = models[0]
trn_errdf = None
tst_errdf = None

def getErrDF(model, window_trn, retdf=None, columns=None):
    for w in window_trn:
        p = model.predict(w[0])
        r = w[1] - p.reshape(w[1].shape)
        r.numpy().sum(axis=1)
        df=pd.DataFrame(r[:,-1], columns=columns) 
        
        if (retdf is None):
            retdf = df
        else:
            retdf.append(df, ignore_index=1)

    return retdf

trn_errdf = getErrDF (model, window_trn100, trn_errdf, df_scaled_trn.columns)
tst_errdf = getErrDF (model, window_tst100, tst_errdf, df_scaled_trn.columns)

In [ ]:
trn_errdf.hist();